In [ ]:
### This notebook is for processing singleSgRNA sequencing data

In [1]:
from os.path import join
from os import listdir
import subprocess
import pandas as pd

In [2]:
input_path = '../../data/sequencing/singleSg/'
fasta_name = 'WGS/00_fastq' # fasta file path
pbs_name = '../../src/crispresso.pbs' # Job file path

In [5]:
# Read the information for the samples
info_file = pd.read_csv(join(input_path,'WGS','SampleInfo.csv'), index_col = 0)

In [6]:
info_file

,Name,Replicate,Day,sgRNA,Amplicon,Amplicon_start_gPos
1,PSMB5,1,5,CAAAGGCATGGGGCTGTCCA,ATCCAACCCCTCCTCACTGTAGTGTCAGCCCAAGGATCATGTGGTT...,23033308
1,VARS1,1,5,CCTTCTCCACCTCGCTGGGA,TGCCACATTCAGGAATCCCCGCTGGGAAGTCAGCTTTCTACAGGGA...,31780747
2,PSMB5,2,5,CAAAGGCATGGGGCTGTCCA,ATCCAACCCCTCCTCACTGTAGTGTCAGCCCAAGGATCATGTGGTT...,23033308
2,CTU2,2,5,CCCTGACTGAGACCCGGACA,CGCTGGTCTGTGTTTCATGCTCTTGGGGTGATGCGGGGAGGGAGCC...,88714482
3,PSMB5,3,5,CAAAGGCATGGGGCTGTCCA,ATCCAACCCCTCCTCACTGTAGTGTCAGCCCAAGGATCATGTGGTT...,23033308
3,CTU2,3,5,CCCTGACTGAGACCCGGACA,CGCTGGTCTGTGTTTCATGCTCTTGGGGTGATGCGGGGAGGGAGCC...,23033308
4,PSMB5,1,17,CAAAGGCATGGGGCTGTCCA,ATCCAACCCCTCCTCACTGTAGTGTCAGCCCAAGGATCATGTGGTT...,23033308
4,VARS1,1,17,CCTTCTCCACCTCGCTGGGA,TGCCACATTCAGGAATCCCCGCTGGGAAGTCAGCTTTCTACAGGGA...,31780747
5,CTU2,1,17,CCCTGACTGAGACCCGGACA,CGCTGGTCTGTGTTTCATGCTCTTGGGGTGATGCGGGGAGGGAGCC...,23033308
6,PSMB5,2,17,CAAAGGCATGGGGCTGTCCA,ATCCAACCCCTCCTCACTGTAGTGTCAGCCCAAGGATCATGTGGTT...,23033308


In [8]:
# Set output folder path
out_path = join(input_path, 'WGS','crispresso_out')

# Compile information for running crispresso in a loop
for i in range(len(info_file)):
    fID = info_file.iloc[i,:].name #fID matches the fastafile name
    
    [Freads,Rreads] = [join(input_path,fasta_name,f) for f in listdir(join(input_path,fasta_name)) if f.split('_')[0] == str(fID)]
    
    name, rep,day, sg, amplicon, amplicon_start= info_file.iloc[i,:]
    
    name_analysis = name+'_D'+str(day)+'_'+str(rep)# set a new name for the output file
    qsub_command = f'qsub -v R1={Freads},R2={Rreads},AMP={amplicon},SG={sg},NAME={name_analysis},OUTPUT={out_path}, -A open {pbs_name}'
    print(qsub_command)
    exit_status = subprocess.call(qsub_command, shell=True)
    if exit_status is 1:  # Check to make sure the job submitted
        print(f'Job {qsub_command} failed to submit')

qsub -v R1=../../data/sequencing/singleSg/WGS/00_fastq/1_R1_001.fastq.gz,R2=../../data/sequencing/singleSg/WGS/00_fastq/1_R2_001.fastq.gz,AMP=ATCCAACCCCTCCTCACTGTAGTGTCAGCCCAAGGATCATGTGGTTGCAGCTTAACTCACCAGGGCCTCTCTTATCCCAGCCACAGATCATGGTGCCCATGGACAGCCCCATGCCTTTGTACTGATACACCATGTTGGCAAGCAGTTTGGAGGCAGCTGCTACAGAGATGCGTTCCTTATTTCGAAGCTCATAGATTCGACATTGCCGAGCCAACAGCCGTTCCCAGAAGCTGCAATCCGCTGCGCCCCCAGCCATGGTGCCTAGCAGGTATGGGTTGATCTCTATCACCTTCTTCACCGTCTGGGAGGCAATGTAA,SG=CAAAGGCATGGGGCTGTCCA,NAME=PSMB5_D5_1,OUTPUT=../../data/sequencing/singleSg/WGS/crispresso_out, -A open ../../src/crispresso.pbs
34444822
qsub -v R1=../../data/sequencing/singleSg/WGS/00_fastq/1_R1_001.fastq.gz,R2=../../data/sequencing/singleSg/WGS/00_fastq/1_R2_001.fastq.gz,AMP=TGCCACATTCAGGAATCCCCGCTGGGAAGTCAGCTTTCTACAGGGAAGAGGCAGGGGGAGGAGCGTCCTCAGCCAGCCCCATCCACGCTGTGCTCCTGCTTAGCCCAGCCCAACCCTCCATACCTGCCCTTCTTTGGCCTTCTCCACCTCGCTGGGATCCAGGTTGCTGTTCAGCAGCTGGTTGTGGAGGCCCTGAGGGTGGAGTGGGAGCAGTCAGGTGGCTGTGACCACAGCCCCACGGCCCTTCCTGGCTGGCCC